In [3]:
import geometry
import numpy as np
import copy

import matplotlib.pyplot as plt


In [ ]:
def load_image(filename):
    # read 3d image and convert to np
    image = plt.imread(filename)
    return image

def slice_image_prompt(image, axis, transform):
    # given the transform, we know the rotation angle
    # we can slice the image at the angle
    # and add prompts (assum there is func for add_prompt)
    
    transformed_image, _ = transform.apply(image, padwith=0)
    # only get the middle slice
    if axis == 0:
        slice = transformed_image[transformed_image.shape[0]//2, :, :]
    elif axis == 1:
        slice = transformed_image[:, transformed_image.shape[1]//2, :]
    else:
        slice = transformed_image[:, :, transformed_image.shape[2]//2]

    
    

    

In [40]:
def slice_and_reconstruct_mask(matrix_3d, transform, axis, num_slices, prompting_points):
    transformed_array, _ = transform.apply_to_array(matrix_3d, padwith=0)

    # get slices indices
    length = transformed_array.shape[axis]
    slice_indices = np.linspace(0, length, num_slices + 2, dtype=int)[1:-1]  

    # initialize mask
    incomplete_mask = np.zeros_like(transformed_array)

    for idx in slice_indices:
        if axis == 0:
            slice = transformed_array[idx, :, :]
        elif axis == 1:
            slice = transformed_array[:, idx, :]
        else: 
            slice = transformed_array[:, :, idx]

        l_intersect_points = set()

        # get list of intersection points on the slice
        for i in range(len(prompting_points) - 1):
            point1 = prompting_points[i]
            point2 = prompting_points[i + 1]

            intersection_point = get_intersection_point(point1, point2, idx, transform)
            l_intersect_points.append(intersection_point)

        if l_intersect_points is not None:
            # mask = get_mask_points(slice, l_intersect_points, model)
            # incomplete_mask[idx, :, :] = mask
            pass
            
    complete_mask = reconstrct_3D_mask(incomplete_mask)
    
    oriented_mask, _ = transform.apply_to_array(complete_mask, padwith=0, inverse=True)


def get_intersection_point(point1, point2, idx, transform):
    rotated_point1 = transform.apply_to_point(point1)
    rotated_point2 = transform.apply_to_point(point2)

    dx = rotated_point2[0] - rotated_point1[0]
    dy = rotated_point2[1] - rotated_point1[1]
    dz = rotated_point2[2] - rotated_point1[2]

    if dx == 0:
        return None

    t = (idx - rotated_point1[0]) / dx

    if 0 <= t <= 1:
        y_intercept = rotated_point1[1] + t * dy
        z_intercept = rotated_point1[2] + t * dz
        return (y_intercept, z_intercept)
    else:
        return None


# given slice, intersection point (on the slice), and segmentation model, get the 2D mask 
def get_mask_points(slice, l_intersect_points, model):
    mask_points = []
    return mask_points


def reconstrct_3D_mask(incomplete_mask):
    complete_mask = np.zeros_like(incomplete_mask)
    return complete_mask

def coord_convert(l_points, x_w, y_h, z_d):
    l_points = np.array(l_points)
    l_points = l_points / [x_w, y_h, z_d] * 2 - 1
    return l_points

